In [1]:
import pandas as pd

file_path = "./data.xlsx"
df = pd.read_excel(file_path)

df.head()

# Remove the first row containing units (now index 0 after reading the file)
df = df.drop(index=0)

# Convert all columns to numeric types, errors='coerce' will replace non-convertible values with NaN
df = df.apply(pd.to_numeric, errors='coerce')

# Display the cleaned dataframe and data types
df.info(), df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1012 entries, 1 to 1012
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   DEPMD          1010 non-null   float64
 1   WOB_AVG        1010 non-null   float64
 2   DEPTVD         1010 non-null   float64
 3   ROPA_AVG       1010 non-null   float64
 4   TORQ_AVG       1010 non-null   float64
 5   SURFRPM_AVG    1010 non-null   float64
 6   MOTORRPM_AVG   1010 non-null   float64
 7   BITRPM_AVG     1010 non-null   float64
 8   SPP_AVG        1010 non-null   float64
 9   CHKP_AVG       1010 non-null   float64
 10  CEMENTP_AVG    1010 non-null   float64
 11  SPM01_AVG      1010 non-null   float64
 12  SPM02_AVG      1010 non-null   float64
 13  SPM03_AVG      1010 non-null   float64
 14  PITACTIVE_AVG  1010 non-null   float64
 15  FLOWIN_AVG     1010 non-null   float64
 16  FLOWOUTP_AVG   1010 non-null   float64
 17  DIN_AVG        1010 non-null   float64
 18  TIN_AVG 

(None,
    DEPMD  WOB_AVG   DEPTVD  ROPA_AVG  TORQ_AVG  SURFRPM_AVG  MOTORRPM_AVG  \
 1  567.0    12.31  2533.49    141.86      4.87         48.0           0.0   
 2  568.0     2.51  2534.33     90.56      4.78         48.0           0.0   
 3  569.0     3.21  2535.19     80.75      4.57         48.0           0.0   
 4  570.0     3.10  2536.04     89.63      4.61         48.0           0.0   
 5  571.0     3.68  2536.87     54.78      4.54         48.0           0.0   
 
    BITRPM_AVG  SPP_AVG  CHKP_AVG  ...  SPM03_AVG  PITACTIVE_AVG  FLOWIN_AVG  \
 1        48.0    689.0       0.0  ...        0.0        227.124       510.0   
 2        48.0    685.0       0.0  ...        0.0        227.451       511.0   
 3        48.0    689.0       0.0  ...        0.0        227.216       510.0   
 4        48.0    685.0       0.0  ...        0.0        227.619       511.0   
 5        48.0    676.0       0.0  ...        0.0        228.554       514.0   
 
    FLOWOUTP_AVG  DIN_AVG  TIN_AVG  DOUT_

In [2]:
import numpy as np

# Constants for conversion
inches_to_feet = 1 / 12
bit_diameter_in = 8.5  # Bit size in inches

# Calculate the cross-sectional area in square feet
AREA_sqft = (np.pi / 4) * (bit_diameter_in * inches_to_feet) ** 2

# Since MOTORRPM_AVG is all zeros, we'll assume BITRPM_AVG as RPM for MSE calculation
# Convert TORQ_AVG from kLbf.ft to Lbf.ft for MSE calculation
df['TORQ_AVG_Lbf.ft'] = df['TORQ_AVG'] * 1000

# Calculate MSE using the given formula
df['MSE'] = (df['WOB_AVG'] * df['BITRPM_AVG'] *
             df['TORQ_AVG_Lbf.ft']) / (df['ROPA_AVG'] * AREA_sqft)

# Check if MSE calculation makes sense by looking at its statistics and first few values
mse_stats = df['MSE'].describe()
mse_head = df['MSE'].head()

mse_stats, mse_head

(count     1010.000000
 mean     18182.367216
 std       8983.546005
 min          0.000000
 25%      11825.114865
 50%      17910.101057
 75%      23584.131339
 max      90190.017744
 Name: MSE, dtype: float64,
 1    51475.794130
 2    16137.683771
 3    22128.638707
 4    19421.609577
 5    37149.899249
 Name: MSE, dtype: float64)

In [3]:
# Calculate the first quartile (Q1) and third quartile (Q3) for HHP
Q1 = df['HHP'].quantile(0.25)
Q3 = df['HHP'].quantile(0.75)
IQR = Q3 - Q1

# Define bin edges based on IQR
bin_edges_iqr = [df['HHP'].min(), Q1 - 1.5 * IQR, Q1, Q3,
                 Q3 + 1.5 * IQR, df['HHP'].max()]
bin_labels_iqr = ['Very Low HHP', 'Low HHP',
                  'Medium HHP', 'High HHP', 'Very High HHP']

# Assign HHP data to bins
df['HHP_Bin_IQR'] = pd.cut(
    df['HHP'], bins=bin_edges_iqr, labels=bin_labels_iqr, include_lowest=True)

# Display the bin edges and the first few entries of the dataframe to verify
bin_edges_iqr, df[['HHP', 'HHP_Bin_IQR']].head()

ValueError: bins must increase monotonically.

In [4]:
# Calculate the first quartile (Q1) and third quartile (Q3) for HHP
Q1 = df['HHP'].quantile(0.25)
Q3 = df['HHP'].quantile(0.75)
IQR = Q3 - Q1

# Display the IQR and quartiles
print("IQR:", IQR)
print("Q1:", Q1)
print("Q3:", Q3)

# Define the lower and upper bounds for the bins
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Define the bin edges
bin_edges = [df['HHP'].min(), lower_bound, Q1, Q3, upper_bound, df['HHP'].max()]

# Display the bin edges
print("Bin Edges:", bin_edges)


IQR: 12.509520182787512
Q1: 72.82939832444782
Q3: 85.33891850723533
Bin Edges: [0.0, 54.065118050266555, 72.82939832444782, 85.33891850723533, 104.10319878141661, 101.2947448591013]


In [7]:
# Calculate Q1 and Q3 using the quantile function
Q1 = df['HHP'].quantile(0.25)
Q3 = df['HHP'].quantile(0.75)

# Calculate the IQR
IQR = Q3 - Q1

# Define the lower and upper bounds for the bins
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Define the bin edges
bin_edges = [lower_bound, Q1, Q3, upper_bound]

print(bin_edges)

# Assign data to bins using pd.cut
df['HHP_Bin'] = pd.cut(df['HHP'], bins=bin_edges, labels=['Very Low HHP', 'Low HHP', 'Medium HHP', 'High HHP', 'Very High HHP'])

# Create a scatter plot of MSE vs ROP with different colors for each HHP bin
import matplotlib.pyplot as plt

colors = {'Very Low HHP': 'red', 'Low HHP': 'orange', 'Medium HHP': 'yellow', 'High HHP': 'green', 'Very High HHP': 'blue'}

fig, ax = plt.subplots()

for label, color in colors.items():
    subset = df[df['HHP_Bin'] == label]
    ax.scatter(subset['ROP_AVG'], subset['MSE'], label=label, color=color)

ax.legend()
ax.grid(True)
plt.xlabel('ROP_AVG')
plt.ylabel('MSE')
plt.show()


[0.0, 54.065118050266555, 72.82939832444782, 85.33891850723533, 104.10319878141661, 101.2947448591013]


In [4]:
import plotly.graph_objs as go
from ipywidgets import widgets, interactive

# Example data
x = [1, 2, 3, 4]
y = [10, 11, 12, 13]

# Create a Plotly figure
fig = go.FigureWidget([go.Scatter(x=x, y=y, mode='markers')])

# Function to update the plot


def update_plot(comment, x_value, y_value):
    with fig.batch_update():
        fig.data[0].x = x
        fig.data[0].y = y
        fig.add_annotation(x=x_value, y=y_value, text=comment,
                           showarrow=True, arrowhead=1)


# Create widgets
comment_text = widgets.Text(value='Comment', description='Comment:')
x_value = widgets.FloatSlider(value=1, min=min(
    x), max=max(x), description='X Value:')
y_value = widgets.FloatSlider(value=10, min=min(
    y), max=max(y), description='Y Value:')

# Display widgets and plot
interactive(update_plot, comment=comment_text,
            x_value=x_value, y_value=y_value)
fig